<a href="https://colab.research.google.com/github/neuroidss/silent_speech/blob/main/silent_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [1]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
if os.path.isdir("/content/drive/MyDrive/silent_speech"):
    %cd "/content/drive/MyDrive/silent_speech"
    if not os.path.isdir("/content/emg_data"):
      !tar -xzf /content/drive/MyDrive/silent_speech/emg_data.tar.gz -C /content
    if not os.path.isdir("/content/text_alignments"):
      !tar -xzf /content/drive/MyDrive/silent_speech/text_alignments.tar.gz -C /content
    %cd /content/drive/MyDrive/silent_speech/nv_wavenet/pytorch
    !python build.py install      
    %cd /content/drive/MyDrive/silent_speech
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !git clone https://github.com/dgaddy/silent_speech.git
    %cd "/content/drive/MyDrive/silent_speech"
    !wget https://zenodo.org/record/4064409/files/emg_data.tar.gz?download=1 -O emg_data.tar.gz
    !tar -xzf /content/drive/MyDrive/silent_speech/emg_data.tar.gz -C /content
    !ln -s /content/emg_data ./emg_data
    !wget https://github.com/dgaddy/silent_speech_alignments/raw/main/text_alignments.tar.gz
    !tar -xzf /content/drive/MyDrive/silent_speech/text_alignments.tar.gz -C /content
    !ln -s /content/text_alignments ./text_alignments
    !git clone https://github.com/NVIDIA/nv-wavenet.git nv_wavenet
    %cd /content/drive/MyDrive/silent_speech/nv_wavenet/pytorch
    !make
    !python build.py install
    %cd /content/drive/MyDrive/silent_speech
    !curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.pbmm
    !curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.scorer


/content/drive/MyDrive/silent_speech


# Environment Setup

In [4]:
!pip install pysndfile
#==1.0.28

In [5]:
!pip install absl-py librosa soundfile matplotlib scipy numba jiwer unidecode deepspeech==0.8.2 praat-textgrids

     |████████████████████████████████| 235 kB 5.2 MB/s 
     |████████████████████████████████| 8.3 MB 42.4 MB/s 
     |████████████████████████████████| 50 kB 3.2 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149854 sha256=c4859430898394e67b5ce84a1de3dceebb665eaa9df1da31abc9dd8eceeafa10
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
  Created wheel for praat-textgrids: filename=praat_textgrids-1.3.1-py3-none-any.whl size=12397 sha256=1534534d0634d66f281688f32e16172d6046ede9581d43a9177045469592ec8b
  Stored in directory: /root/.cache/pip/wheels/78/5e/7c/ebd0d2a251e76cc196edcb3781be75b68dfbf91f149060e6d5
Successfully built python-Levenshtein praat-textgrids


# Running

In [ ]:
#!python wavenet_model.py --output_directory "./models/wavenet_model/" --silent_data_directories ""
!python wavenet_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_1_of_50.pt" --output_directory "./models/wavenet_model/" --silent_data_directories ""

['wavenet_model.py', '--pretrained_wavenet_model', './models/wavenet_model/wavenet_model_1_of_50.pt', '--output_directory', './models/wavenet_model/', '--silent_data_directories', '']
output example: (./emg_data/voiced_parallel_data/5-6, 194)
/content/drive/MyDrive/silent_speech/nv_wavenet/pytorch/wavenet.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  self.conv.weight, gain=torch.nn.init.calculate_gain(w_init_gain))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:

#!python transduction_model.py --data_size_fraction 0.5 --batch_size 1 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 2 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 32 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"

!python transduction_model.py --start_training_from ./models/transduction_model/model.pt --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"


In [ ]:
!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"


In [13]:
!python evaluate.py --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output


Common confusions (confusion, accuracy)
eh ah 16.2 37.7
l ah 17.4 33.1
er ah 17.8 32.9
iy ah 18.2 30.5
ah ae 18.3 34.5
r ah 18.4 33.4
t ah 18.7 27.5
ay ah 20.1 43.4
ih ah 22.5 31.5
n ah 23.8 35.4
/content/drive/MyDrive/silent_speech/nv_wavenet/pytorch/wavenet.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  self.conv.weight, gain=torch.nn.init.calculate_gain(w_init_gain))
TensorFlow: v2.2.0-24-g1c1b2b9
DeepSpeech: v0.8.2-0-g02e4c76
2022-02-03 23:52:10.294596: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
targets: ['henderson he called you saw that shooting star last night', 'he read and reread the paper fearing the worst had happened to me', 'he heard footsteps running to and fro in the rooms and up and down stairs behind him', 'some of the refugees were exchanging news with the people on the omnibuses', 'such things i told myself could 

In [14]:
!python evaluate.py --testset_file testset_origdev.json --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output


Common confusions (confusion, accuracy)
eh ah 16.2 37.7
l ah 17.4 33.1
er ah 17.8 32.9
iy ah 18.2 30.5
ah ae 18.3 34.5
r ah 18.4 33.4
t ah 18.7 27.5
ay ah 20.1 43.4
ih ah 22.5 31.5
n ah 23.8 35.4
/content/drive/MyDrive/silent_speech/nv_wavenet/pytorch/wavenet.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  self.conv.weight, gain=torch.nn.init.calculate_gain(w_init_gain))
TensorFlow: v2.2.0-24-g1c1b2b9
DeepSpeech: v0.8.2-0-g02e4c76
2022-02-04 00:17:52.829284: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
targets: ['henderson he called you saw that shooting star last night', 'he read and reread the paper fearing the worst had happened to me', 'he heard footsteps running to and fro in the rooms and up and down stairs behind him', 'some of the refugees were exchanging news with the people on the omnibuses', 'such things i told myself could 